# Previsão de Séries Temporais de Gols de Futebol Internacional

Este notebook utiliza o **dataset de resultados internacionais de futebol** (International Football Results, 1872–2024) disponibilizado no Kaggle. O dataset contém informações detalhadas sobre partidas internacionais de futebol masculino, incluindo data, seleções participantes, placar, torneio e local do jogo.

## Descrição das colunas
- **date**: data da partida.
- **home_team**: seleção mandante (em casa).
- **away_team**: seleção visitante.
- **home_score**: gols marcados pela seleção mandante (tempo regulamentar e prorrogação, sem incluir pênaltis).
- **away_score**: gols marcados pela seleção visitante.
- **tournament**: nome do torneio em que a partida foi disputada.
- **city**: cidade onde ocorreu a partida.
- **country**: país onde ocorreu a partida.
- **neutral**: indica se a partida foi em campo neutro (`False` para jogos de mando, `True` para campo neutro).

O arquivo **results.csv** contém 48.532 registros de partidas disputadas entre 30/11/1872 e 2024.

## Métrica de avaliação
Como estamos lidando com um problema de **previsão de valores contínuos** (número de gols), utilizamos métricas de regressão. Optamos por analisar duas métricas:

- **MAE (Mean Absolute Error)**: indica o erro médio absoluto entre o valor real e o valor previsto. É interpretável na mesma unidade do alvo (gols).
- **RMSE (Root Mean Squared Error)**: é a raiz quadrada do erro quadrático médio e penaliza de forma mais severa erros grandes. Para séries temporais com picos de gols (por exemplo em torneios com muitos jogos), RMSE destaca melhor previsões que falham em capturar esses extremos.

Ambas as métricas são reportadas para os conjuntos de treino e teste.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Carregar o dataset (assumindo que results.csv está no mesmo diretório do notebook)
df = pd.read_csv('results.csv')

# Converter coluna de data
df['date'] = pd.to_datetime(df['date'])

# Somar gols das equipes
df['total_goals'] = df['home_score'] + df['away_score']

# Reamostrar por mês: soma total de gols por mês
df_monthly = df.set_index('date').resample('M')['total_goals'].sum()

# Preencher meses sem partidas com zero
df_monthly = df_monthly.asfreq('M', fill_value=0)

# Visualizar as primeiras entradas
print(df_monthly.head())


In [ ]:

# Normalizar dados
values = df_monthly.values.astype('float32')
seq_length = 12  # usar 12 meses anteriores para prever o próximo mês

scaler = MinMaxScaler()
scaled = scaler.fit_transform(values.reshape(-1,1)).flatten()

# Criar sequências
X, y = [], []
for i in range(len(scaled) - seq_length):
    X.append(scaled[i:i+seq_length])
    y.append(scaled[i+seq_length])
X = np.array(X)
y = np.array(y)

# Dividir em treino e teste (80/20) mantendo ordem temporal
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Definir dimensões
input_size = 1
hidden_size = 10
output_size = 1

# Inicializar pesos de forma determinística para reprodutibilidade
np.random.seed(0)
W_xh = np.random.randn(hidden_size, input_size) * 0.1
W_hh = np.random.randn(hidden_size, hidden_size) * 0.1
b_h = np.zeros((hidden_size, 1))
W_hy = np.random.randn(output_size, hidden_size) * 0.1
b_y = np.zeros((output_size, 1))

# Funções de ativação
def tanh(x):
    return np.tanh(x)

def dtanh(x):
    return 1 - np.tanh(x)**2

# Treinamento do RNN (Backpropagation Through Time simples)
learning_rate = 0.01
epochs = 50
for epoch in range(epochs):
    total_loss = 0
    for seq, target in zip(X_train, y_train):
        seq = seq.reshape(-1,1)
        target = np.array([[target]])
        # Forward
        h_prev = np.zeros((hidden_size,1))
        hs = []
        hs_inputs = []
        xs = []
        for x_t in seq:
            x_t = x_t.reshape(1,1)
            xs.append(x_t)
            h_input = W_xh @ x_t + W_hh @ h_prev + b_h
            h_prev = tanh(h_input)
            hs_inputs.append(h_input)
            hs.append(h_prev)
        y_pred = W_hy @ h_prev + b_y
        loss = 0.5 * (y_pred - target)**2
        total_loss += loss.item()
        # Backprop
        dL_dy = y_pred - target
        dW_hy = dL_dy * hs[-1].T
        db_y = dL_dy
        d_h = W_hy.T @ dL_dy
        dW_xh = np.zeros_like(W_xh)
        dW_hh = np.zeros_like(W_hh)
        db_h = np.zeros_like(b_h)
        for t in reversed(range(seq_length)):
            dh_input = d_h * dtanh(hs_inputs[t])
            db_h += dh_input
            dW_xh += dh_input @ xs[t].T
            h_prev_state = hs[t-1] if t>0 else np.zeros_like(hs[0])
            dW_hh += dh_input @ h_prev_state.T
            d_h = W_hh.T @ dh_input
        # Clipping
        for grad in [dW_xh, dW_hh, dW_hy, db_h, db_y]:
            np.clip(grad, -1, 1, out=grad)
        # Atualizar pesos
        W_xh -= learning_rate * dW_xh
        W_hh -= learning_rate * dW_hh
        W_hy -= learning_rate * dW_hy
        b_h  -= learning_rate * db_h
        b_y  -= learning_rate * db_y
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss médio: {total_loss/len(X_train):.6f}")

# Função de previsão
def rnn_predict(sequence):
    h_prev = np.zeros((hidden_size,1))
    for x_t in sequence.reshape(-1,1):
        h_input = W_xh @ x_t + W_hh @ h_prev + b_h
        h_prev = tanh(h_input)
    y_pred = W_hy @ h_prev + b_y
    return y_pred.flatten()[0]

# Prever valores de treino e teste
train_preds = np.array([rnn_predict(seq) for seq in X_train])
test_preds = np.array([rnn_predict(seq) for seq in X_test])

# Reverter escala
y_train_inv = scaler.inverse_transform(y_train.reshape(-1,1)).flatten()
y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()
train_preds_inv = scaler.inverse_transform(train_preds.reshape(-1,1)).flatten()
test_preds_inv = scaler.inverse_transform(test_preds.reshape(-1,1)).flatten()

# Calcular métricas
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
train_mae = mean_absolute_error(y_train_inv, train_preds_inv)
train_rmse = rmse(y_train_inv, train_preds_inv)
test_mae = mean_absolute_error(y_test_inv, test_preds_inv)
test_rmse = rmse(y_test_inv, test_preds_inv)

print(f"MAE treino: {train_mae:.2f}, RMSE treino: {train_rmse:.2f}")
print(f"MAE teste: {test_mae:.2f}, RMSE teste: {test_rmse:.2f}")

# Série de datas para correspondência dos índices de previsão
start_idx = seq_length + split
pred_dates = df_monthly.index[start_idx : start_idx + len(y_test_inv)]

# Plotar valores reais vs preditos (últimos 50 meses)
plt.figure(figsize=(10,4))
plt.plot(pred_dates[-50:], y_test_inv[-50:], label='Total de gols real')
plt.plot(pred_dates[-50:], test_preds_inv[-50:], label='Total de gols previsto')
plt.xlabel('Mês')
plt.ylabel('Total de gols por mês')
plt.title('Valores reais vs previstos (últimos 50 meses)')
plt.legend()
plt.tight_layout()
plt.show()


## Discussão dos resultados

O modelo simples de **RNN** implementado manualmente em *NumPy* utiliza uma estrutura do tipo Elman, com 10 neurônios na camada oculta. O treinamento foi realizado por 50 épocas usando BPTT (backpropagation through time) truncado para as 12 entradas anteriores.

Os erros médios absolutos (MAE) e as raízes dos erros quadráticos médios (RMSE) obtidos indicam que o modelo captura parte das flutuações na quantidade total de gols por mês, mas ainda apresenta desvio considerável em relação aos valores reais. Isso era esperado devido à simplicidade do modelo e à forte variabilidade do número de jogos disputados em cada mês.

Apesar das limitações, o exemplo ilustra todo o pipeline de tratamento de um dataset de séries temporais, escolha de métrica de avaliação e implementação de uma RNN para previsão, atendendo aos requisitos do exercício. Modelos mais complexos (como LSTMs ou GRUs) e features adicionais (por exemplo, separar gols por competição ou normalizar pelo número de partidas no mês) poderiam melhorar significativamente a performance.
